In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('Doctor_Asiful_Sk.csv')

In [ ]:
df.head()

In [ ]:
# Encoding categorical variables
le_symptom = LabelEncoder()
df['Symptom_encoded'] = le_symptom.fit_transform(df['Symptom'])

le_medicine = LabelEncoder()#Encon
df['Medicine_encoded'] = le_medicine.fit_transform(df['Prescriptions'])

le_notes = LabelEncoder()
df['Notes_encoded'] = le_notes.fit_transform(df['Notes'])

le_dignosis = LabelEncoder()
df['Dignosis_encoded'] = le_dignosis.fit_transform(df['Dignosis'])

In [ ]:
def data_split(data, ratio):
    np.random.seed(42)
    shuffled = np.random.permutation(len(data))
    test_set_size = int(len(data) * ratio)
    test_indices = shuffled[:test_set_size]
    train_indices = shuffled[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train, test = data_split(df, 0.2)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
X_train = train[['Symptom_encoded']].to_numpy()
X_test = test[['Symptom_encoded']].to_numpy()

Y_train = train[['Medicine_encoded']].to_numpy()
Y_test = test[['Medicine_encoded']].to_numpy()

y_train = train[['Notes_encoded']].to_numpy()
y_test = test[['Notes_encoded']].to_numpy()

z_train = train[['Dignosis_encoded']].to_numpy()
z_test = test[['Dignosis_encoded']].to_numpy()


In [ ]:
model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model1.fit(X_train, y_train)

In [ ]:
model2 = RandomForestClassifier(n_estimators=100, random_state=42)
model2.fit(X_train, Y_train)

In [ ]:

model3 = RandomForestClassifier(n_estimators=100, random_state=42)
model3.fit(X_train, z_train)

In [ ]:
# Assume symptoms_list is a list of symptoms
symptoms_list = ['Bleeding piles','Vomiting']

# Initialize lists to store predictions for each symptom
predicted_notes_encoded_list = []
predicted_medicine_encoded_list = []
predicted_diagnosis_encoded_list = []

# Loop through each symptom
for symptom in symptoms_list:
    # Encode the symptom
    new_symptom_encoded = le_symptom.transform([symptom])
    
    # Predict notes, medicine, and diagnosis for the symptom
    predicted_notes_encoded = model1.predict([new_symptom_encoded])
    predicted_medicine_encoded = model2.predict([new_symptom_encoded])
    predicted_diagnosis_encoded = model3.predict([new_symptom_encoded])
    
    # Append predictions to lists
    predicted_notes_encoded_list.append(predicted_notes_encoded)
    predicted_medicine_encoded_list.append(predicted_medicine_encoded)
    predicted_diagnosis_encoded_list.append(predicted_diagnosis_encoded)

# Decode the predictions
predicted_notes = le_notes.inverse_transform(predicted_notes_encoded_list)
predicted_medicine = le_medicine.inverse_transform(predicted_medicine_encoded_list)
predicted_diagnosis = le_dignosis.inverse_transform(predicted_diagnosis_encoded_list)

# Now you have lists of predicted notes, medicine, and diagnosis for each symptom in symptoms_list


In [ ]:
print(f"Predicted Medicine: {predicted_medicine[0]}")
print(f"predicted_notes: {predicted_notes}")
print(f"predicted_dignosis: {predicted_diagnosis}")

In [ ]:
notes = []
medicine = []
diagnosis = []

for i in range(len(predicted_medicine)):
    if 'nan' in predicted_medicine[i]:
        notes.append(predicted_notes[i])
        medicine.append(predicted_medicine[i])
        diagnosis.append(predicted_diagnosis[i])
    else:
        notes.append(predicted_notes[i])
        # Check if predicted_medicine[i] is a string before splitting
        if isinstance(predicted_medicine[i], str):
            medicine.extend(predicted_medicine[i].split(';'))
        else:
            medicine.append(predicted_medicine[i])
        diagnosis.append(predicted_diagnosis[i])

# Ensure all arrays have the same length
max_length = max(len(notes), len(medicine), len(diagnosis))
notes += [''] * (max_length - len(notes))
medicine += [''] * (max_length - len(medicine))
diagnosis += [''] * (max_length - len(diagnosis))

# Create DataFrame with all values
dataframe2 = pd.DataFrame({
    'Medicine': medicine,
    'Notes': notes,
    'Diagnosis': diagnosis
})

In [ ]:
dataframe2